In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.layers import *

In [2]:
df = pd.read_csv("../input/bank-customer-churn-modeling/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.drop('RowNumber', axis=1, inplace=True)

In [5]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df.drop(['CustomerId', 'Surname'], axis=1, inplace=True)

In [10]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
X = df.iloc[:, 0: 10]
y = df.iloc[:,10]
print(X.shape, y.shape)

(10000, 10) (10000,)


In [13]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


# Preprocessing

In [14]:
X['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [15]:
X.Geography.replace({'France':0, 'Spain': 1, 'Germany':2}, inplace=True)
X.Geography.unique()

array([0, 1, 2])

In [16]:
X.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [17]:
X.Gender.replace({'Male':0, 'Female':1}, inplace=True)
X.Gender.unique()

array([1, 0])

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [22]:
scaler.data_min_

array([350.  ,   0.  ,   0.  ,  18.  ,   0.  ,   0.  ,   1.  ,   0.  ,
         0.  ,  11.58])

In [20]:
X = pd.DataFrame(scaler.transform(X))
X.head()

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


,0,1,2,3,4,5,6,7,8,9
0,-0.698924,0.00,1.0,-0.238860,0.02,0.000000,-0.333333,1.0,1.0,-0.000055
1,-0.698968,0.25,1.0,-0.239043,0.01,0.000001,-0.333333,0.0,1.0,-0.000055
2,-0.699392,0.00,1.0,-0.238860,0.08,0.000003,-0.111111,1.0,0.0,-0.000055
3,-0.698604,0.00,1.0,-0.239408,0.01,0.000000,-0.222222,0.0,0.0,-0.000056
4,-0.698000,0.25,1.0,-0.238678,0.02,0.000002,-0.333333,1.0,1.0,-0.000056


# Keep 9000 for training and 1000 for testing

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(9000, 10) (9000,) (1000, 10) (1000,)


# Buildinig a neural network using keras seuqential model

In [27]:
model = keras.Sequential()
model.add(Input(10,))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=6)

Epoch 1/6
282/282 [==============================] - 1s 2ms/step - loss: 0.6314 - accuracy: 0.6562
Epoch 2/6
282/282 [==============================] - 0s 1ms/step - loss: 0.4942 - accuracy: 0.7949
Epoch 3/6
282/282 [==============================] - 0s 2ms/step - loss: 0.4826 - accuracy: 0.7949
Epoch 4/6
282/282 [==============================] - 0s 2ms/step - loss: 0.4796 - accuracy: 0.7949
Epoch 5/6
282/282 [==============================] - 0s 2ms/step - loss: 0.4789 - accuracy: 0.7949
Epoch 6/6
282/282 [==============================] - 0s 2ms/step - loss: 0.4785 - accuracy: 0.7949


In [32]:
model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 1ms/step - loss: 0.4570 - accuracy: 0.8090


[0.45696279406547546, 0.8090000152587891]

Model gives a reasonable accuracy of 80% on both training and testing